In [87]:
import numpy as np
import pandas as pd
import kagglehub
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st

In [60]:
# Download latest version
path = kagglehub.dataset_download("amirmotefaker/ab-testing-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/kirill/.cache/kagglehub/datasets/amirmotefaker/ab-testing-dataset/versions/1


In [61]:
control = pd.read_csv(path + '/control_group.csv', sep=';')
test = pd.read_csv(path + '/test_group.csv', sep=';')

In [62]:
control = control.rename(columns={'Campaign Name': 'Campaign_Name',
                                  'Spend [USD]':'Spend_USD',
                                  '# of Impressions':'n_of_Impressions',
                                  '# of Website Clicks': 'n_of_Website_Clicks',
                                  '# of Searches':'n_of_Searches',
                                  '# of View Content':'n_of_View_Content',
                                  '# of Add to Cart':'n_of_Add_to_Cart',
                                  '# of Purchase': 'n_of_Purchase'})

test = test.rename(columns={'Campaign Name': 'Campaign_Name',
                                  'Spend [USD]':'Spend_USD',
                                  '# of Impressions':'n_of_Impressions',
                                  '# of Website Clicks': 'n_of_Website_Clicks',
                                  '# of Searches':'n_of_Searches',
                                  '# of View Content':'n_of_View_Content',
                                  '# of Add to Cart':'n_of_Add_to_Cart',
                                  '# of Purchase': 'n_of_Purchase'})

In [63]:
total = pd.concat([control, test])

In [64]:
total.Date = pd.to_datetime(total.Date, format = '%d.%m.%Y')

In [65]:
total = total.sort_values(by = 'Date')

In [71]:
total = total.query('Date != "2019-08-05"')

In [72]:
total = total.reset_index()

In [73]:
del total['index']

#### Определим **целевые** и **прокси** метрики для анализа эффективности маркетинговой компании:  
##### Целевые метрики:  
1. Количество покупок
2. Конверсия в оплату (покупки/охват)


##### Прокси метрики:  
1. Общее число кликов
2. Конверсия из показов в клики
3. Число пользователей, добавивших товар в корзину
4. Промежуточные коневрсии (Клики - Просмотр контента, Просмотры - добавление в корзину)
5. Стоимость привлечения пользователя (общие затраты на кампанию / количество покупок)

In [75]:
metrics = total.groupby(['Campaign_Name'], as_index=False) \
     .agg({'Spend_USD':'sum',
           'n_of_Impressions':'sum',
           'Reach':'sum',
           'n_of_Website_Clicks':'sum',
           'n_of_View_Content':'sum',
           'n_of_Add_to_Cart':'sum',
           'n_of_Purchase':'sum'})

In [80]:
metrics['CR_pay'] = metrics.n_of_Purchase / metrics.Reach
metrics['CR_view_to_add_to_cart'] = metrics.n_of_Add_to_Cart / metrics.n_of_View_Content
metrics['CR_clicks_to_view'] = metrics.n_of_View_Content / metrics.n_of_Website_Clicks
metrics['CR_impr_to_click'] = metrics.n_of_Website_Clicks / metrics.n_of_Impressions
metrics['cost_per_user'] = metrics.Spend_USD / metrics.n_of_Purchase
metrics['n_not_Purchase'] = metrics.Reach - metrics.n_of_Purchase

In [81]:
metrics_1 = metrics[['Campaign_Name', 'n_of_Purchase', 'n_not_Purchase','CR_pay', 'n_of_Website_Clicks', 'n_of_Add_to_Cart', 'CR_clicks_to_view', 'CR_view_to_add_to_cart', 'cost_per_user']]

In [83]:
metrics_1

,Campaign_Name,n_of_Purchase,n_not_Purchase,CR_pay,n_of_Website_Clicks,n_of_Add_to_Cart,CR_clicks_to_view,CR_view_to_add_to_cart,cost_per_user
0,Control Campaign,15161.0,2561342.0,0.005884,154303.0,37700.0,0.36532,0.668795,4.407229
1,Test Campaign,14869.0,1494740.0,0.009850,175107.0,25490.0,0.31342,0.464451,5.016814


#### Для А/В тестирования выбираем целевую метрику конверсию в покупку, а также общее число покупок.

##### **Конверсия в покупку**

In [133]:
chi_test_cr_pay = metrics_1[['n_of_Purchase', 'n_not_Purchase']]

In [134]:
chi_test_cr_pay

,n_of_Purchase,n_not_Purchase
0,15161.0,2561342.0
1,14869.0,1494740.0


In [155]:
stat, p, dof, expected = st.chi2_contingency(chi_test_cr_pay)
print(f"Значение статистики = {stat:.4f}")
print(f"p-value = {p:.4f}")
print(f"Степени свободы = {dof}")
print("Ожидаемые значения:\n", expected)

Значение статистики = 2051.0053
p-value = 0.0000
Степени свободы = 1
Ожидаемые значения:
 [[  18935.45382261 2557567.54617739]
 [  11094.54617739 1498514.45382261]]


### ДАЛЕЕ ДЕЛАЮ ОПИСАТЕЛЬНЫЙ АНАЛИЗ ВСЕХ ОСТАЛЬНЫХ МЕТРИК, А ТАКЖЕ ВОРОНКУ ДЕЙСТВИЙ ПОЛЬЗОВАТЕЛЯ ДЛЯ АНАЛИЗА КОНВЕРСИИ НА КАЖДОМ ЭТАПЕ.